## Linear

Importamos os módulos essenciais do PyTorch e do NumPy.

- **torch.nn (neural networks):** para construir a arquitetura do modelo.
- **torch.optim (optimizer):** para definir o algoritmo de otimização.
- **torch e numpy:** para manipulação de tensores e dados numéricos.

In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim

Criamos um conjunto de dados simples que segue a equação de uma linha reta, y=3x+2, com a adição de um pequeno ruído para simular dados do mundo real.

- **torch.linspace(0, 1, 100):** Gera 100 pontos igualmente espaçados entre 0 e 1 para a nossa variável de entrada x.
- **.unsqueeze(1):** Adiciona uma dimensão extra aos tensores. Isso é necessário porque os modelos do PyTorch esperam dados em formato de "batch" (lotes), então um vetor de 100 elementos precisa se tornar uma matriz de 100x1.
- **0.1 * torch.randn(x.size()):** Adiciona ruído aleatório (distribuição normal) para que o modelo tenha que "aprender" a linha de tendência, em vez de apenas ajustar perfeitamente aos pontos.

In [2]:
# Dados sintéticos para regressão linear
x = torch.linspace(0, 1, 100).unsqueeze(1)
y = 3 * x + 2 + 0.1 * torch.randn(x.size())  # y = 3x + 2 + ruído

Esta é a parte central da configuração do treinamento.

- **model = nn.Linear(1, 1):** Cria uma camada linear.
    - O primeiro 1 indica que há uma característica de entrada (x).
    - O segundo 1 indica que há uma característica de saída (y).
- Este modelo possui dois parâmetros internos que serão aprendidos: o peso (weight) e o viés (bias), correspondendo ao coeficiente angular e ao intercepto da linha.
- **criterion = nn.MSELoss():** Define a Função de Perda (ou "critério"). Neste caso, usamos o Erro Quadrático Médio (MSE), que mede a diferença média quadrada entre as previsões do modelo e os valores reais. O objetivo do treinamento é minimizar essa perda.
- **optimizer = optim.SGD(model.parameters(), lr=0.1):** Define o Otimizador. Usamos o Gradiente Descendente Estocástico (SGD).
    - **model.parameters():** Informa ao otimizador quais parâmetros do modelo ele deve otimizar (ajustar).
    - **lr=0.1:** Define a taxa de aprendizado (learning rate), que controla o tamanho do passo que o otimizador dá a cada iteração para ajustar os parâmetros.

In [3]:

# Modelo linear
model = nn.Linear(1, 1)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

Este é o ciclo principal onde o modelo aprende a partir dos dados. O processo é repetido por um número fixo de épocas.

- **Passo 1 (Forward):** O modelo faz uma previsão (pred) para os dados de entrada x.
- **Passo 2 (Loss):** A função de perda calcula quão "ruim" a previsão foi, gerando um valor de loss.
- **Passo 3 (Zero Grad):** Zera os gradientes das iterações anteriores, para evitar acumulação.
- **Passo 4 (Backward):** Esta é a "mágica do autograd". PyTorch calcula os gradientes (derivadas) da loss em relação aos parâmetros do modelo (peso e viés).
- **Passo 5 (Optimizer Step):** O otimizador usa os gradientes para ajustar os parâmetros do modelo, movendo-os na direção que minimiza a perda.

In [4]:
for epoch in range(200):
    # 1. Passo para frente (Forward pass):
    pred = model(x) # Faz uma previsão usando o modelo atual

    # 2. Cálculo da perda:
    loss = criterion(pred, y) # Compara a previsão com os valores reais

    # 3. Zera os gradientes:
    optimizer.zero_grad() # Limpa os gradientes de épocas anteriores

    # 4. Passo para trás (Backward pass / Backpropagation):
    loss.backward() # Calcula os gradientes da perda em relação aos parâmetros

    # 5. Passo do otimizador:
    optimizer.step() # Atualiza os parâmetros (peso e viés) usando os gradientes calculados

Os resultados serão:

In [5]:
print("Regressão Linear - Peso e Viés:", model.weight.item(), model.bias.item())

Regressão Linear - Peso e Viés: 2.8562865257263184 2.080226421356201


_____

## Não linear

Em vez de uma linha reta, criamos dados que seguem uma função seno com ruído. Isso exige um modelo mais complexo do que uma simples linha.

- **torch.sin(2 * np.pi * x):** Cria uma curva senoidal.
- **0.1 * torch.randn(x.size()):** Adiciona ruído para tornar o problema de aprendizado mais realista.
- A entrada x é a mesma do exemplo de regressão linear.

In [6]:
# Dados para regressão não-linear
y_nl = torch.sin(2 * np.pi * x) + 0.1 * torch.randn(x.size())

Aqui, a arquitetura do modelo é significativamente mais complexa do que uma única camada linear.

- **nn.Sequential:** É um contêiner que permite empilhar camadas em sequência, onde a saída de uma camada se torna a entrada da próxima.
- **nn.Linear(1, 64):** A primeira camada "esconde" (hidden layer) recebe a entrada de 1 dimensão e a expande para 64 dimensões.
- **nn.Tanh():** Esta é uma função de ativação não-linear. A inclusão de funções não-lineares entre as camadas é o que permite à rede neural aprender e modelar relações não-lineares, como a curva senoidal. Sem elas, o modelo seria apenas uma combinação de transformações lineares, equivalente a uma única camada linear.
- **nn.Linear(64, 64):** Uma segunda camada oculta que mantém a dimensionalidade em 64.
- **nn.Linear(64, 1):** A camada de saída que recebe as 64 dimensões e as mapeia de volta para a saída de 1 dimensão (y_nl).

In [7]:
# Modelo não-linear (MLP)
model_nl = nn.Sequential(
    nn.Linear(1, 64),
    nn.Tanh(),
    nn.Linear(64, 64),
    nn.Tanh(),
    nn.Linear(64, 1)
)

Usamos o otimizador Adam, que geralmente converge mais rápido e é mais robusto do que o SGD para a maioria dos problemas.

- **optim.Adam:** Um otimizador mais avançado que ajusta a taxa de aprendizado para cada parâmetro individualmente.
- **model_nl.parameters():** Informa ao otimizador para otimizar todos os parâmetros de todas as camadas do modelo.
- **lr=0.01:** A taxa de aprendizado é um pouco menor, o que é comum com o otimizador Adam.

In [8]:
optimizer_nl = optim.Adam(model_nl.parameters(), lr=0.01)

O loop de treinamento segue o mesmo padrão de "Forward -> Loss -> Backward -> Step" do exemplo de regressão linear, mas com mais épocas devido à complexidade da função a ser aprendida.

In [9]:
for epoch in range(1000):
    pred_nl = model_nl(x)
    loss_nl = criterion(pred_nl, y_nl)
    optimizer_nl.zero_grad()
    loss_nl.backward()
    optimizer_nl.step()

Os resultados serão:

In [10]:
print("Regressão Não Linear - Loss final:", loss_nl.item())

Regressão Não Linear - Loss final: 0.009081622585654259
